# Forecasting Danube Water Levels

## Introduction
Hi! 
Welcome to the water level forecasting benchmark exercise for the Danube's measuring station "Kienstock" at km 2015.21. 
There's an official [government forecast](https://www.noel.gv.at/wasserstand/#/de/Messstellen/Details/207357/WasserstandPrognose/48Stunden) which we aim to beat with an LSTM model. 
enjoy ;-)

## Let's first make sure we have the right versions of Tensorflow and Keras

In [ ]:
!pip install --upgrade keras-applications keras-preprocessing setuptools tensorflow==1.14.0 keras==2.2.5

In [ ]:
import tensorflow
if not tensorflow.__version__ == '1.14.0':
    print(tensorflow.__version__)
    raise ValueError('please upgrade to TensorFlow 1.14.0, or restart your Kernel (Kernel->Restart & Clear Output)')

import keras
if not keras.__version__ == '2.2.5':
    print(keras.__version__)
    raise ValueError('please upgrade to Keras 2.2.5, or restart your Kernel (Kernel->Restart & Clear Output)')

OK, now let's get the Keras elements we'll use.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import LSTM
from keras.callbacks import Callback

## Now let's load all other dependencies

In [ ]:
import numpy as np # linear algebra
from numpy import concatenate

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pandas import read_csv
from pandas import DataFrame

import matplotlib.pyplot as plt # plotting
from mpl_toolkits.mplot3d import Axes3D

import sklearn # ML libraries
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression

import os # accessing directory structure
# import sys # maybe we need this to access the file system

%matplotlib inline

Let's see if we have acces to the data file

In [ ]:
!rm * # clear current working directory

In [ ]:
!wget https://emelt2.ocloud.de/index.php/s/aSS6kGMeTGXP0GW/download
!mv download danube-waterlevel-Kienstock_2002-2019.csv

There is 1 csv file in the current version of the dataset:


In [ ]:
# check what files we have got
for dirname, _, filenames in os.walk('.'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

The next hidden code cells define functions for plotting data. Click on the "Code" button in the published kernel to reveal the hidden code.

In [ ]:
# Distribution graphs (histogram/bar graph) of column data
def plotPerColumnDistribution(df, nGraphShown, nGraphPerRow):
    nunique = df.nunique()
    df = df[[col for col in df if nunique[col] > 1 and nunique[col] < 50]] # For displaying purposes, pick columns that have between 1 and 50 unique values
    nRow, nCol = df.shape
    columnNames = list(df)
    nGraphRow = (nCol + nGraphPerRow - 1) / nGraphPerRow
    plt.figure(num = None, figsize = (6 * nGraphPerRow, 8 * nGraphRow), dpi = 80, facecolor = 'w', edgecolor = 'k')
    for i in range(min(nCol, nGraphShown)):
        plt.subplot(nGraphRow, nGraphPerRow, i + 1)
        columnDf = df.iloc[:, i]
        if (not np.issubdtype(type(columnDf.iloc[0]), np.number)):
            valueCounts = columnDf.value_counts()
            valueCounts.plot.bar()
        else:
            columnDf.hist()
        plt.ylabel('counts')
        plt.xticks(rotation = 90)
        plt.title(f'{columnNames[i]} (column {i})')
    plt.tight_layout(pad = 1.0, w_pad = 1.0, h_pad = 1.0)
    plt.show()


In [ ]:
# Correlation matrix
def plotCorrelationMatrix(df, graphWidth):
    filename = df.dataframeName
    df = df.dropna('columns') # drop columns with NaN
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    if df.shape[1] < 2:
        print(f'No correlation plots shown: The number of non-NaN or constant columns ({df.shape[1]}) is less than 2')
        return
    corr = df.corr()
    plt.figure(num=None, figsize=(graphWidth, graphWidth), dpi=80, facecolor='w', edgecolor='k')
    corrMat = plt.matshow(corr, fignum = 1)
    plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
    plt.yticks(range(len(corr.columns)), corr.columns)
    plt.gca().xaxis.tick_bottom()
    plt.colorbar(corrMat)
    plt.title(f'Correlation Matrix for {filename}', fontsize=15)
    plt.show()


In [ ]:
# Scatter and density plots
def plotScatterMatrix(df, plotSize, textSize):
    df = df.select_dtypes(include =[np.number]) # keep only numerical columns
    # Remove rows and columns that would lead to df being singular
    df = df.dropna('columns')
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    columnNames = list(df)
    if len(columnNames) > 10: # reduce the number of columns for matrix inversion of kernel density plots
        columnNames = columnNames[:10]
    df = df[columnNames]
    ax = pd.plotting.scatter_matrix(df, alpha=0.75, figsize=[plotSize, plotSize], diagonal='kde')
    corrs = df.corr().values
    for i, j in zip(*plt.np.triu_indices_from(ax, k = 1)):
        ax[i, j].annotate('Corr. coef = %.3f' % corrs[i, j], (0.8, 0.2), xycoords='axes fraction', ha='center', va='center', size=textSize)
    plt.suptitle('Scatter and Density Plot')
    plt.show()


Now we're ready to read in the data and use the plotting functions to visualize the data.

### Let's check 1st file: danube-waterlevel-Kienstock_2002-2019.csv

In [ ]:
nRowsRead = None # specify 'None' if want to read whole file
# danube-waterlevel-Kienstock_2002-2019.csv may have more rows in reality, but we are only loading/previewing the first 1000 rows
df1 = pd.read_csv('danube-waterlevel-Kienstock_2002-2019.csv', delimiter=',', nrows = nRowsRead)
df1.dataframeName = 'danube-waterlevel-Kienstock_2002-2019.csv'
nRow, nCol = df1.shape
print(f'There are {nRow} rows and {nCol} columns')

Let's take a quick look at what the data looks like:

In [ ]:
df1.head(5)

Distribution graphs (histogram/bar graph) of sampled columns:

In [ ]:
plotPerColumnDistribution(df1, 3, 1)

In [ ]:
df1.hist()

Correlation matrix:

In [ ]:
plotCorrelationMatrix(df1, 8)

Scatter and density plots:

In [ ]:
plotScatterMatrix(df1, 9, 10)

Let's continue with max values only. We are looking for flood maxima afte all, and min and mean are highly correlated.

In [ ]:
df1 = df1.loc[:, ["date", "max"]]
df1.head(5)

In [ ]:
df1_plot = df1.iloc[:,1:2].values.astype(float)
# Visualising the Data
plt.plot(df1_plot, color = 'blue', label = 'Water Level @ Kienstock (km 2015)')
plt.title('Danube Water Level Historical Data')
plt.xlabel('Time (Days)')
plt.ylabel('max level in cm')
plt.legend()
plt.show()

Let's see if we have any missing data

In [ ]:
missing_data = df1.isnull()

In [ ]:
for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("") 

In case there are any "Trues" we replace the NaNs with the mean of the neighbors

In [ ]:
for i in range (0, len(df1)):
    if missing_data.iloc[i,1]:
        df1.iloc[i,1] = (df1.iloc[i-1,1] + df1.iloc[i+1,1]) / 2
        print(df1.iloc[i,1])

Let's check again

In [ ]:
missing_data = df1.isnull()

In [ ]:
for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("") 

Next we have to prepare the data sets

In [ ]:
# defining some parameters
batchsize = 64 # needed for LSTM
timesteps = 8 # window size
test_share = 0.1 # test set size

In [ ]:
def get_set_length(length, batch, timesteps):
    # substract test_percent to be excluded from training, reserved for testset
    modulo = int((length - 2 * timesteps) % batch)
    return int(length - modulo)

In [ ]:
# setting data set lengths
length_tot = len(df1)
print(length_tot)
length_train = get_set_length((length_tot * (1 - test_share)), batchsize, timesteps)
print(length_train)
length_test = get_set_length((length_tot - length_train), batchsize, timesteps)
print(length_test)

In [ ]:
# creating data sets with feature scaling between 0 and 1.

sc = MinMaxScaler(feature_range = (0, 1))
total_set = df1.iloc[:,1:2].values
total_set_scaled = sc.fit_transform(np.float64(total_set))

training_set_scaled = total_set_scaled[0:length_train]
print(training_set_scaled.shape)
print(training_set_scaled[0:2])

test_set = total_set[length_train+1:length_train+length_test]
print(test_set.shape)
print(test_set[0:2])

test_set_scaled = total_set_scaled[length_train+1:length_train+length_test]
print(test_set_scaled.shape)
print(test_set_scaled[0:2])

Now let's set a baseline forecast with linear regression

In [ ]:
# Creating data structures for linear regression with n timesteps
lr_train_windows = []
for i in range(timesteps, length_train): 
    lr_train_windows.append(training_set_scaled[i-timesteps:i,0])

print(lr_train_windows[0:2])
print(np.array(lr_train_windows).shape)

lr_test_windows = []
for i in range(timesteps, length_test): 
    lr_test_windows.append(test_set_scaled[i-timesteps:i,0])

print(lr_test_windows[0:2])
print(np.array(lr_test_windows).shape)

In [ ]:
print(test_set_scaled[timesteps-1:timesteps+1])

In [ ]:
y_test_orig = test_set[timesteps-1:length_test-1]
print(y_test_orig.shape)

In [ ]:
# convert data structures to np.arrays for linear regression
x_y_train = np.asarray(lr_train_windows)
x_train = x_y_train[:,:timesteps-1]
y_train = x_y_train[:,timesteps-1:timesteps]

In [ ]:
# do the fit
lr_model = LinearRegression()
lr_model.fit(x_train, y_train)
# the regression coefficients
print ('Coefficients: ', lr_model.coef_)

In [ ]:
# get predicted data on the training set (for the visualization)
y_train_pred = lr_model.predict(x_train) 

# inverse transform (reverse feature scaling)
y_train_pred_unscaled = sc.inverse_transform(y_train_pred)

In [ ]:
print(y_train_pred_unscaled[0:2])

In [ ]:
# convert test data structures to numpy arrays
x_y_test = np.asarray(lr_test_windows)
x_test = x_y_test[:,:timesteps-1]
y_test = x_y_test[:,timesteps-1:timesteps]

In [ ]:
print(x_test[0:3])

In [ ]:
print(y_test[0:3])

In [ ]:
print(y_test_orig[0:4])

In [ ]:
y_test_unscaled = sc.inverse_transform(y_test)
print(y_test_unscaled[0:4])

In [ ]:
# get predicted data on the test set
y_test_pred = lr_model.predict(x_test)

# inverse transform (reverse feature scaling)
y_test_pred_unscaled = sc.inverse_transform(y_test_pred)

print(y_test_pred_unscaled[0:4])

In [ ]:
# calculate mean squared error on the test set predictions
test_residuals = y_test_pred_unscaled - y_test_orig
lr_rmse = np.sqrt(np.sum(np.power(test_residuals,2)) / len(test_residuals))
print('RMSE = %.2f' % lr_rmse)

In [ ]:
# calculate mean squared error on the trivial predictions
test_residuals = y_test_orig[1:] - y_test_orig[:-1]
lr_rmse = np.sqrt(np.sum(np.power(test_residuals,2)) / len(test_residuals))
print('RMSE = %.2f' % lr_rmse)

In [ ]:
# Visualising the results
plt.plot(y_test_orig[100:125], color = 'blue', label = 'Real Water Level')
plt.plot(y_test_pred_unscaled[100:125], color = 'green', label = 'Predicted Water Level')
plt.plot(y_test_orig[99:124], color = 'red', label = 'Trivially Predicted Water Level')
plt.title('Water Level at Kienstock km 2015')
plt.xlabel('Time')
plt.ylabel('max level in cm')
plt.legend()
plt.show()